In [1]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import seaborn as sns

In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
st=stopwords.words('english')
stem=PorterStemmer()
def stemming(txt):
    txt=txt.split()
    txt=[stem.stem(x) for x in txt if x not in st]
    return ' '.join(txt)
def preprocess(txt):
    txt=re.sub('@[A-Za-z0-9]+',' ',txt)
    txt=re.sub('[^a-z\s]',' ',txt.lower())
    txt=re.sub('\s+',' ',txt)
    
    return txt


In [4]:
df=pd.read_csv('train_hate_speech.csv')


In [5]:
df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...
24778,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,6,0,6,0,1,youu got wild bitches tellin you lies


In [6]:
df=df[['tweet','class']]


In [7]:
df['class'].value_counts()


class
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [8]:
df['pre_process']=df['tweet'].apply(preprocess)
df['pre_process']=df['pre_process'].map(stemming)

C:\Users\himan\AppData\Local\Temp\ipykernel_6104\2157959576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pre_process']=df['tweet'].apply(preprocess)
C:\Users\himan\AppData\Local\Temp\ipykernel_6104\2157959576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pre_process']=df['pre_process'].map(stemming)


In [9]:
df

,tweet,class,pre_process
0,!!! RT @mayasolovely: As a woman you shouldn't...,2,rt woman complain clean hous amp man alway tak...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1,rt boy dat cold tyga dwn bad cuffin dat hoe st...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,rt dawg rt ever fuck bitch start cri confus shit
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1,rt g anderson base look like tranni
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,rt shit hear might true might faker bitch told ya
...,...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1,muthaf lie pearl emanuel right tl trash mine b...
24779,"you've gone and broke the wrong heart baby, an...",2,gone broke wrong heart babi drove redneck crazi
24780,young buck wanna eat!!.. dat nigguh like I ain...,1,young buck wanna eat dat nigguh like aint fuck...
24781,youu got wild bitches tellin you lies,1,youu got wild bitch tellin lie


In [10]:
tok=Tokenizer(oov_token='unknown')
tok.fit_on_texts(df['pre_process'])
train_x,test_x,train_y,test_y=train_test_split(df['pre_process'],df['class'],stratify=df['class'])


In [11]:
trainx=tok.texts_to_sequences(train_x)
testx=tok.texts_to_sequences(test_x)

train_pad=pad_sequences(trainx,maxlen=15)
test_pad=pad_sequences(testx,maxlen=15)


In [12]:
trainx

[[1039, 58, 8, 127, 95, 420],
 [31, 1259, 14, 24, 445, 91, 13, 1116, 815, 2],
 [3, 1238, 7717, 3567, 5727, 6328, 4],
 [895, 4, 34, 14, 1491],
 [2420, 2, 10, 2702, 118, 43, 13, 476, 8, 13, 476, 8],
 [242, 7, 2, 90, 40, 1670, 283, 1349, 1484],
 [3, 1273, 235, 623, 1696, 2272, 3, 1641, 7845, 38, 408, 360, 162, 934, 366],
 [2862, 164, 1228, 26, 5598, 3239, 5598, 81, 1350, 2],
 [3, 233, 21, 7, 8, 93, 341],
 [3, 14832, 3, 831, 786, 2, 10, 3, 39, 355, 451, 1596],
 [3, 796, 270, 38, 341, 33, 2],
 [268, 70, 1183, 8],
 [2, 203, 86, 188, 543, 15, 253, 1196],
 [770, 11, 2],
 [7065, 2],
 [2, 67, 9, 773],
 [8, 95, 180, 2],
 [79, 1043, 762],
 [3, 61, 77, 2, 50, 1366],
 [5012, 167, 9, 4, 12, 2, 9, 9852, 12, 84, 4],
 [393, 42, 451, 146, 157, 1262, 2242, 2674, 2, 160, 2668, 13390],
 [3, 2799, 137, 1090, 224, 137, 16, 16393, 4358, 208, 5],
 [3, 1863, 2, 14, 355, 33, 3897],
 [74, 2, 129],
 [11, 184, 30, 1272, 23, 40],
 [3, 530, 4],
 [13320, 249, 86, 258, 4, 86, 37],
 [19, 269, 2],
 [52, 202, 17],
 [575, 2

In [13]:
len(trainx)

18587

In [14]:
testx

[[2, 15, 798, 1134, 10555, 3243],
 [888, 1073, 2068, 4, 79, 383, 1050, 12, 32, 10],
 [15, 1121, 10, 170, 31, 301, 50, 1662, 36, 702],
 [3, 2444, 321, 59, 2, 19, 11, 142, 1721, 4464, 79, 38, 2030, 679],
 [2, 956, 404],
 [3, 1779, 2229, 1004, 36, 5, 6, 17252],
 [27, 1624, 182, 12309, 328, 4],
 [83, 35, 3474, 7998, 114, 142, 656, 362],
 [2932, 204, 2],
 [39, 13, 3560, 2968, 13121],
 [160, 77, 118, 24, 2],
 [115, 9, 3480, 30],
 [6928, 106, 16],
 [922, 2, 112, 883, 3, 652, 167, 493, 562],
 [1556, 272, 7, 703, 88],
 [6276, 2, 467],
 [24, 4, 43],
 [1922, 26, 109, 5, 6, 9283, 2360, 3491, 12, 2],
 [121, 2, 13081, 154, 11, 695, 273],
 [2899, 27, 5131, 72],
 [3,
  7883,
  687,
  107,
  435,
  1254,
  15,
  568,
  33,
  2,
  6889,
  1439,
  17,
  16050,
  12,
  10,
  60],
 [2284, 28, 84],
 [1382, 28, 2, 780, 59, 2466, 39, 4550, 8176],
 [3, 26, 359, 350, 19, 31, 29, 4],
 [7190,
  3288,
  7191,
  7,
  1053,
  5478,
  1700,
  1908,
  425,
  5479,
  337,
  7192,
  2399,
  4025,
  383,
  5,
  6,
  76,


In [15]:
len(testx)

6196

In [16]:
train_pad

array([[   0,    0,    0, ...,  127,   95,  420],
       [   0,    0,    0, ..., 1116,  815,    2],
       [   0,    0,    0, ..., 5727, 6328,    4],
       ...,
       [   0,    0,    0, ..., 2816, 6538,    2],
       [   0,    0,    0, ...,  126,   48,  544],
       [   0,    3,  738, ...,  266,  706,  163]])

In [17]:
test_pad

array([[    0,     0,     0, ...,  1134, 10555,  3243],
       [    0,     0,     0, ...,    12,    32,    10],
       [    0,     0,     0, ...,  1662,    36,   702],
       ...,
       [    0,     0,     0, ...,    30,   767,  8315],
       [    0,     0,     0, ...,  1129, 16260,   796],
       [    0,     0,     0, ...,   251,  6688,  2032]])

In [18]:
len(train_pad[0])

15

In [19]:
len(test_pad[0])

15

In [20]:
len(train_pad)

18587

In [21]:
len(test_pad)

6196

In [22]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(train_pad,train_y)


KNeighborsClassifier()

In [23]:
from sklearn.metrics import accuracy_score,confusion_matrix


In [24]:
ans=knn.predict(test_pad)


In [25]:
accuracy_score(test_y,ans)


0.7540348612007747

In [26]:
print(confusion_matrix(test_y,ans))


[[  14  332   11]
 [  62 4537  199]
 [  19  901  121]]


In [27]:
from tensorflow.keras.layers import Dense,Embedding,Flatten
from tensorflow.keras.models import Sequential


In [28]:
m=Sequential()
m.add(Embedding(input_dim=24784,input_length=15,output_dim=64))
m.add(Dense(32))
m.add(Flatten())
m.add(Dense(3,activation='softmax'))
m.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
m.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 64)            1586176   
                                                                 
 dense (Dense)               (None, 15, 32)            2080      
                                                                 
 flatten (Flatten)           (None, 480)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1443      
                                                                 
Total params: 1589699 (6.06 MB)
Trainable params: 1589699 (6.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
hist=m.fit(train_pad,train_y,batch_size=64,epochs=5)


Epoch 1/5


291/291 [==============================] - 4s 11ms/step - loss: 0.4800 - accuracy: 0.8282
Epoch 2/5
291/291 [==============================] - 4s 15ms/step - loss: 0.2217 - accuracy: 0.9227
Epoch 3/5
291/291 [==============================] - 3s 11ms/step - loss: 0.1302 - accuracy: 0.9553
Epoch 4/5
291/291 [==============================] - 3s 10ms/step - loss: 0.0823 - accuracy: 0.9725
Epoch 5/5
291/291 [==============================] - 3s 10ms/step - loss: 0.0570 - accuracy: 0.9818


In [30]:
ans1=m.predict(test_pad)


194/194 [==============================] - 0s 2ms/step


In [31]:
ans1

array([[2.07316084e-03, 9.97841358e-01, 8.54991304e-05],
       [8.91731815e-06, 9.99991059e-01, 1.49615181e-10],
       [1.01194136e-01, 8.98805857e-01, 4.76379078e-08],
       ...,
       [2.74244841e-04, 2.11917283e-03, 9.97606516e-01],
       [1.24055706e-03, 8.89243841e-01, 1.09515645e-01],
       [1.12734549e-03, 9.49035883e-01, 4.98367958e-02]], dtype=float32)

In [32]:
ans1=np.argmax(ans1,axis=1)

In [33]:
ans1

array([1, 1, 1, ..., 2, 1, 1], dtype=int64)

In [34]:
print('With ANN score {}'.format(accuracy_score(test_y,ans1)*100))
print('With KNN score {}'.format(accuracy_score(test_y,ans)*100))

With ANN score 87.57262750161394
With KNN score 75.40348612007747
